# Importazione librerie

In [7]:
import pandas as pd
import os

# Importazione e conversione datasets

Data la grande dimensione di alcuni dei dataset utilizzati, questi verranno convertiti da `.csv` a `.parquet` (Apache Parquet Format)

In [8]:
#df = pd.read_csv('./Datasets/Dati_sensori_meteo_2021.csv', low_memory=False)
#df.to_parquet('./Datasets/Dati_sensori_meteo_2021.parquet')

Il codice sotto funziona ma riempie la RAM

In [9]:
#dtypes = {'IdSensore': 'string', 'Valore':'string'}
#parse_dates
#meteo = pd.concat(map(pd.read_parquet, [f'{path}Dati_sensori_meteo_2021_parte_1.parquet', f'{path}Dati_sensori_meteo_2021_parte_2.parquet']))
#meteo.head()

Importazione datasets convertiti

In [10]:
path = './Datasets/'

#### Rilevazioni sensori metereologiche
##### Attributi: 
- IdSensore: 
    - Tipologia: Testo normale
- Data: 
    - Data e ora, Marcatura oraria flessibile
- Valore: 
    - Tipologia: numero
    - Legenda: 9999 = dato mancante 888, 8888 = direzione vento variabile 777, 7777 = calma (solo per direzione di vento)
- idOperatore	
    - LEGENDA: 1: Valore medio 3: Valore massimo 4: Valore cumulato (per la pioggia)
    - Testo normale
- Stato	
    - LEGENDA: VA, VV = dato valido NA, NV, NC = dato invalido NI = dato incerto ND = dato non disponibile
    - Testo normale

In [49]:
meteo_21 = pd.read_parquet(f'{path}data_meteo/Dati_sensori_meteo_2021.parquet')
meteo_21.head()

,IdSensore,Data,Valore,Stato,idOperatore
0,5857,01/01/2021 00:00:00,0.0,VA,4
1,5864,01/01/2021 00:00:00,2.0,VA,1
2,6158,01/01/2021 00:00:00,84.8,VA,1
3,6431,01/01/2021 00:00:00,0.0,VA,1
4,10585,01/01/2021 00:00:00,0.0,VA,4


#### Rilevazioni sensori qualità dell'aria
##### Attributi: 
- IdSensore: 
    - Identificativo Univoco che distingue il sensore
    - Tipologia: Testo normale
- Data: 
    - Data e ora, Marcatura oraria flessibile
- Valore: 
    - Tipologia: numero
    - LEGENDA:-9999 = dato mancante o invalido
- idOperatore	
    - LEGENDA: 1: Valore medio 3: Valore massimo 4: Valore cumulato (per la pioggia)
    - Testo normale
- Stato	
    - VA = dato valido NA = dato invalido I dati presenti in questo archivio relativi agli ultimi 3-6 mesi, contengono ancora valori incerti che possono subire modifiche da parte degli operatori delle reti (invalidazione manuale). Il processo di validazione dei dati prevede una fase di valutazione finale che si conclude entro il 30.3 dell’anno successivo a quello di misura.
    - Testo normale

In [13]:
aria_21 = pd.read_csv(f'{path}Dati_sensori_aria_2021.csv')
aria_21.head()

,IdSensore,Data,Valore,Stato,idOperatore
0,5832,01/01/2021 12:00:00 AM,-9999.0,NaN,1
1,5832,01/01/2021 04:00:00 AM,-9999.0,NaN,1
2,6328,01/01/2021 03:00:00 AM,-9999.0,NaN,1
3,5504,01/01/2021 04:00:00 AM,-9999.0,NaN,1
4,5504,01/01/2021 03:00:00 AM,-9999.0,NaN,1


#### Sensori metereologici
##### Attributi: 
- IdSensore
    - Foreign Key con vincolo di integrità referenziale a `IdSensore` di meteo_21
    - Testo normale
- Tipologia	
    - Grandfezza misurata
    - Testo normale
- Unità DiMisura	
    - Unità di misura della grandezza
    - Testo normale
- IdStazione	
    - Numero identificativo della stazione (penso perchè ogni stazione può avere più sensori)
    - Testo normale
- NomeStazione	
    - Località della stazione (Spesso è 'Comune via' o 'Comune località)'
    - Testo normale
- Quota	
    - Altitudine
    - Numero
- Provincia
    - Sigla della provincia
    - Testo normale
- DataStart	
    - Data e ora
- DataStop	
    - Data e ora
- Storico	
    - Noto che assume valori N e S ma non capisco cosa sia
    - Testo normale
- UTM_Nord	
    - Coordinata UTM nord (le cordinate utm permettono di individuare univocamente un punto sulla cartina terrestre)
    - Testo normale
- UTM_Est	
    - Coordinata UTM est
    - Testo normale
- lng	
    - Longitudine (senza simbolo gradi)
    - Numero
- lat	
    - Latitudine (senza simbolo gradi)
    - Numero
- location	
    - (latitudine°, longitudine°)
    - Posizione
    
##### Grandezze disponibili
Sono disponibili i dati delle grandezze:
- Livello Idrometrico (cm)
- Altezza neve (cm)
- Precipitazione (mm)
- Temperatura (°C)
- Umidità Relativa (%)
- Radiazione Globale (W/m2)
- Velocità e Direzione Vento (m/s e gradi).
- Velocità e direzione del vento (m/s e gradi N) raffica
NB: l’orario del dato è "ora solare" e si riferisce alle osservazioni ottenute fino all’orario indicato.

In [14]:
sensori_meteo = pd.read_csv(f'{path}Stazioni_Meteorologiche.csv', dtype = {'Tipologia': 'category'})
sensori_meteo.head()
#sensori_meteo['Tipologia'].value_counts()

,IdSensore,Tipologia,Unità DiMisura,IdStazione,NomeStazione,Quota,Provincia,DataStart,DataStop,Storico,UTM_Nord,UTM_Est,lng,lat,location
0,22006,Umidità Relativa,%,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
1,22007,Direzione Vento,°,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
2,22003,Temperatura,°C,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
3,22009,Velocità Vento,m/s,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
4,17612,Altezza Neve,cm,1545,San Siro Alpe Rescascìa,1285,CO,07/11/2019,NaN,N,5102069,518430,9.238323,46.071947,"(46.071947417956004, 9.238323435918229)"


#### Sensori qualità dell'aria
##### Attributi: 
- IdSensore
    - Foreign Key con vincolo di integrità referenziale a `IdSensore` di aria_21
    - Testo normale
- NomeTipoSensore	
    - Grandezza misurata
    - Testo normale
- Unità DiMisura	
    - Unità di misura della grandezza
    - Testo normale
- IdStazione	
    - Numero identificativo della stazione (penso perchè ogni stazione può avere più sensori)
    - Testo normale
- NomeStazione	
    - Località della stazione (Spesso è 'Comune via' o 'Comune località)'
    - Testo normale
- Quota	
    - Altitudine
    - Numero
- Provincia
    - Sigla della provincia
    - Testo normale
- DataStart	
    - Data e ora
- DataStop	
    - Data e ora
- Storico	
    - Noto che assume valori N e S ma non capisco cosa sia
    - Testo normale
- UTM_Nord	
    - Coordinata UTM nord (le cordinate utm permettono di individuare univocamente un punto sulla cartina terrestre)
    - Testo normale
- UTM_Est	
    - Coordinata UTM est
    - Testo normale
- lng	
    - Longitudine (senza simbolo gradi)
    - Numero
- lat	
    - Latitudine (senza simbolo gradi)
    - Numero
- location	
    - (latitudine°, longitudine°)
    - Posizione

In [43]:
sensori_aria = pd.read_csv(f'{path}Stazioni_qualita_dell_aria.csv', dtype = {'NomeTipoSensore': 'category'})
sensori_aria.head()
#sensori_aria['NomeTipoSensore'].value_counts()

,IdSensore,NomeTipoSensore,UnitaMisura,Idstazione,NomeStazione,Quota,Provincia,Comune,Storico,DataStart,DataStop,Utm_Nord,UTM_Est,lat,lng,location
0,17286,Biossido di Azoto,µg/m³,1374,Monza Parco,181.0,MB,Monza,N,01/05/2013,NaN,5049780,521452,45.601232,9.275073,"(45.60123219955139, 9.27507265782848)"
1,17284,Ammoniaca,µg/m³,1374,Monza Parco,181.0,MB,Monza,N,08/02/2013,NaN,5049780,521452,45.601232,9.275073,"(45.60123219955139, 9.27507265782848)"
2,17290,PM10 (SM2005),µg/m³,1374,Monza Parco,181.0,MB,Monza,N,15/02/2013,NaN,5049780,521452,45.601232,9.275073,"(45.60123219955139, 9.27507265782848)"
3,17285,Ossidi di Azoto,µg/m³,1374,Monza Parco,181.0,MB,Monza,N,01/05/2013,NaN,5049780,521452,45.601232,9.275073,"(45.60123219955139, 9.27507265782848)"
4,17288,Ozono,µg/m³,1374,Monza Parco,181.0,MB,Monza,N,09/02/2013,NaN,5049780,521452,45.601232,9.275073,"(45.60123219955139, 9.27507265782848)"


In [44]:
sensori_aria.dtypes

IdSensore             int64
NomeTipoSensore    category
UnitaMisura          object
Idstazione            int64
NomeStazione         object
Quota               float64
Provincia            object
Comune               object
Storico              object
DataStart            object
DataStop             object
Utm_Nord              int64
UTM_Est               int64
lat                 float64
lng                 float64
location             object
dtype: object

#### Zonizzazione

In [46]:
zonizzazione = pd.read_csv(f'{path}zonizzazione/zonizzazione_ABCD.csv', dtype = {'Zona': 'category'})
zonizzazione.head()
#sum(zonizzazione['Residenti 2008'])
#zonizzazione['Zona'].value_counts()

,Provincia,Codice Istat,Comune,Zona,Residenti 2008,Superficie (ha)
0,BG,16009,AMBIVERE,A,2341,327
1,BG,16013,ARZAGO D'ADDA,A,2836,944
2,BG,16018,BAGNATICA,A,4119,639
3,BG,16020,BARIANO,A,4396,714
4,BG,16021,BARZANO',A,5178,356


# Analisi esplorativa

- verificare quali siano tutte le quantità uniche rilevate dai sensori qualità dell'aria
- quali siano i gruppi di inquinanti principali
- quali siano i limiti di assunzione umana (gionralieri e annuali)
    - Vedi [qui](https://www.arpalombardia.it/Pages/Aria/Inquinanti.aspx) per limiti e piccola descrizione
    - Vedi [qui](https://www.regione.lombardia.it/wps/portal/istituzionale/HP/DettaglioRedazionale/servizi-e-informazioni/cittadini/salute-e-prevenzione/Sicurezza-negli-ambienti-di-vita-e-di-lavoro/inquinamento-atmosferico/inquinamento-atmosferico/) per approfondimento

In [47]:
sensori_aria['NomeTipoSensore'].value_counts()

Biossido di Azoto             166
Ossidi di Azoto               166
Biossido di Zolfo             125
Monossido di Carbonio         101
Ozono                          91
PM10 (SM2005)                  90
Particolato Totale Sospeso     55
Particelle sospese PM2.5       40
Benzene                        32
Nikel                          15
Piombo                         15
Arsenico                       15
Cadmio                         15
Benzo(a)pirene                 15
Ammoniaca                      14
PM10                            4
BlackCarbon                     4
Monossido di Azoto              3
Name: NomeTipoSensore, dtype: int64

- Si potrebbero considerare solo gli inquinanti **più importanti e con più stazioni**
- Ovvero **Biossido di Azoto e altri ossidi di azoto | PM10 (SM2005) e PM2,5 e/o Particolato totale | Ozono troposferico | Monossido di carbonio | Bisossido di Zolfo | Benzene**

### Riduzione dimensioni

Meteo: 343 stazioni con 1262 sensori

Aria: 174 stazioni con 966 sensori

Zonizzazione: 9 826 141 abitanti in 1545 comuni divisi in 7 o 8 zone

- in `stazioni_aria` e `stazioni_meteo` tenere solo righe con `Stato == VA` (facoltativo)
- da `stazioni_aria` e `stazioni_meteo` trovare solo gli `IdSensore` di tipologia utile
- tenere solo stazioni meteo dove ci sono stazioni aria (joint su `lat` e `lng`?)
- inner joint tra `aria_xx` e `meteo_xx` con stazioni filtrate
- left joint (non case sensitive) con dataset `zonizzazione` rispetto a `Comune` tenendo solo la colonna `Zona`
- salvare i nuovi dataset in formato parquet

In [48]:
stazioni_aria = sensori_aria.groupby(['NomeStazione', 'Comune'], as_index = False).count()
pd.merge(stazioni_aria, zonizzazione, left_on = stazioni_aria['Comune'].str.lower(), right_on = zonizzazione['Comune'].str.lower(), how='left')

,key_0,NomeStazione,Comune_x,IdSensore,NomeTipoSensore,UnitaMisura,Idstazione,Quota,Provincia_x,Storico,...,UTM_Est,lat,lng,location,Provincia_y,Codice Istat,Comune_y,Zona,Residenti 2008,Superficie (ha)
0,abbadia cerreto,Abbadia Cerreto v. Fanfulla,Abbadia Cerreto,4,4,4,4,4,4,4,...,4,4,4,4,LO,98001.0,ABBADIA CERRETO,B,283.0,616.0
1,abbiategrasso,Abbiategrasso v. Serafino Dell'Uomo,Abbiategrasso,3,3,3,3,3,3,3,...,3,3,3,3,MI,15002.0,ABBIATEGRASSO,A,31578.0,4677.0
2,agrate brianza,Agrate Brianza v.Ferrario,Agrate Brianza,5,5,5,5,5,5,5,...,5,5,5,5,MB,108001.0,AGRATE BRIANZA,AGG MI,14962.0,1125.0
3,arconate,Arconate v. De Gasperi,Arconate,4,4,4,4,4,4,4,...,4,4,4,4,MI,15007.0,ARCONATE,A,6406.0,835.0
4,arese,Arese v. Matteotti,Arese,6,6,6,6,6,6,6,...,6,6,6,6,MI,15009.0,ARESE,AGG MI,19496.0,657.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,villasanta,Villasanta v. A. Volta,Villasanta,5,5,5,5,5,5,5,...,5,5,5,5,MB,108049.0,VILLASANTA,AGG MI,13530.0,484.0
170,vimercate,Vimercate v. dell'Ospedale,Vimercate,7,7,7,7,7,7,7,...,7,7,7,7,MB,108050.0,VIMERCATE,AGG MI,25643.0,2062.0
171,vobarno,Vobarno v.Sottostrada,Vobarno,1,1,1,1,1,1,1,...,1,1,1,1,BS,17204.0,VOBARNO,C,8169.0,5331.0
172,voghera,Voghera v.Pozzoni,Voghera,6,6,6,6,6,6,6,...,6,6,6,6,PV,18182.0,VOGHERA,B,39802.0,6335.0
